# 📘 Anleitung: Starten des Notebooks

Dieses Notebook wird von GitHub geladen. Um die Analyse zu starten, folgen Sie bitte diesen Schritten:

1. **Umgebung vorbereiten:** Gehen Sie oben im Menü auf **Laufzeit** (Runtime) → **Alle ausführen** (Run all).
2. **Warnung bestätigen:** Wenn das Fenster *"Warnung: Dieses Notebook wurde nicht von Google erstellt"* erscheint, klicken Sie auf **„Trotzdem ausführen“**.
3. **Berechnung starten:** Sobald das Bedienfeld erscheint, klicken Sie unten auf die grüne Schaltfläche **▶ Ausführen**.

---
*Die Warnung ist eine Google-Sicherheitsmaßnahme für GitHub-Dateien. Die grüne Schaltfläche trainiert das KI-Modell und sendet die Prognosen an die App.*

In [ ]:
# @title
# --- Sticky UI Panel (responsive, mobile friendly) ---
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

display(HTML("""
<style>
/* Sticky container */
#lotto_sticky_bar {
  position: sticky;
  top: 0;
  z-index: 9999;
  background: white;
  border: 1px solid #e5e7eb;
  border-radius: 14px;
  padding: 12px;
  margin-bottom: 10px;
  box-shadow: 0 8px 24px rgba(0,0,0,0.06);
}

/* Make notebook width nicer on desktop */
.output-area { max-width: 1100px; }

/* -------- Responsive tweaks -------- */
.lotto-row {
  display: flex;
  gap: 10px;
  flex-wrap: wrap;              /* allow wrap on small screens */
  align-items: center;
}

.lotto-row > .lotto-btn {
  flex: 0 0 auto;               /* default: keep size */
}

.lotto-row > .lotto-progress {
  flex: 1 1 320px;              /* progress takes remaining width */
  min-width: 240px;
}

.lotto-tabs .widget-tab {
  width: 100%;
}

/* Mobile: stack everything, full-width buttons */
@media (max-width: 520px) {
  #lotto_sticky_bar { padding: 10px; border-radius: 12px; }
  .lotto-row { flex-direction: column; align-items: stretch; }
  .lotto-row > .lotto-btn,
  .lotto-row > .lotto-progress {
    width: 100% !important;
    min-width: 0 !important;
  }

  /* Make buttons big & readable */
  .lotto-row button {
    height: 46px !important;
    font-size: 15px !important;
    white-space: nowrap !important;   /* do not wrap text */
  }

  /* Progress label smaller so it fits */
  .lotto-row .widget-label {
    font-size: 12px !important;
  }

  /* Tabs + outputs a bit taller on mobile */
  .lotto-output {
    height: 300px !important;
  }
  .lotto-logs {
    height: 300px !important;
    font-size: 13px !important;
  }

  /* Title smaller */
  .lotto-title-main { font-size: 16px !important; }
  .lotto-title-sub  { font-size: 12px !important; }
}
</style>
"""))

# ----- Widgets -----
title = widgets.HTML(
    value="""
    <div>
      <div class="lotto-title-main" style="font-size:18px;font-weight:800">LOTTO 6aus49 – Runner</div>
      <div class="lotto-title-sub" style="font-size:12.5px;color:#4b5563">
        Klicke auf ▶ Ausführen. Ergebnisse werden mit deinem Account synchronisiert.
      </div>
    </div>
    """
)

btn_run = widgets.Button(
    description="▶ Ausführen",
    button_style="success",
    tooltip="Startet den kompletten Lauf",
    layout=widgets.Layout(width="170px", height="40px")
)
btn_run.add_class("lotto-btn")

btn_clear = widgets.Button(
    description="🧹 Clear",
    tooltip="Leert Ergebnis/Logs",
    layout=widgets.Layout(width="120px", height="40px")
)
btn_clear.add_class("lotto-btn")

progress = widgets.IntProgress(value=0, min=0, max=100, description="Fortschritt:", bar_style="info")
progress.add_class("lotto-progress")

status = widgets.HTML(value="")

out_result = widgets.Output(layout={
    "border":"1px solid #e5e7eb",
    "border_radius":"12px",
    "padding":"10px",
    "height":"260px",
    "overflow":"auto"
})
out_result.add_class("lotto-output")

log_area = widgets.Textarea(
    value="",
    placeholder="Logs erscheinen hier…",
    layout=widgets.Layout(width="100%", height="260px")
)
log_area.add_class("lotto-logs")

tabs = widgets.Tab(children=[out_result, log_area])
tabs.set_title(0, "Ergebnis")
tabs.set_title(1, "Logs")

tabs_box = widgets.VBox([tabs])
tabs_box.add_class("lotto-tabs")

controls = widgets.HBox(
    [btn_run, btn_clear, progress],
    layout=widgets.Layout(gap="10px")
)
controls.add_class("lotto-row")

# Sticky container + UI
display(HTML('<div id="lotto_sticky_bar"></div>'))
display(widgets.VBox([title, controls, status, tabs_box], layout=widgets.Layout(gap="10px")))

# ----- Helper -----
def ui_set_status(html, kind="info"):
    colors = {
        "info":   ("#1f2937", "#eef2ff", "#c7d2fe"),
        "success":("#065f46", "#ecfdf5", "#a7f3d0"),
        "warn":   ("#92400e", "#fffbeb", "#fcd34d"),
        "error":  ("#991b1b", "#fef2f2", "#fecaca"),
    }
    fg, bg, bd = colors.get(kind, colors["info"])
    status.value = f"""
    <div style="margin-top:10px;padding:10px 12px;border-radius:12px;
                border:1px solid {bd}; background:{bg}; color:{fg}; font-size:13px">
      {html}
    </div>
    """

def log(msg):
    log_area.value = (log_area.value + msg + "\n") if log_area.value else (msg + "\n")

def clear_all(_=None):
    progress.value = 0
    status.value = ""
    log_area.value = ""
    with out_result:
        clear_output()

btn_clear.on_click(clear_all)
ui_set_status("Bereit. Klicke auf <b>▶ Ausführen</b>.", "info")

In [ ]:
# Imports
import os
import io
import time
import json
import zipfile
import requests
import urllib.parse as up
from datetime import datetime
import numpy as np
import pandas as pd
import pytz
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [ ]:
# Konfiguration (Backend)
BACKEND_URL_DEFAULT = "https://apilotto.euroceiling39.ru"

# Modellparameter (wie im Original)
model_params = {
    "batch_size": 98,
    "epochs": 123,
    "window_length": 24
}

# ----------------------------
# Hilfsfunktionen
# ----------------------------
def download_and_parse_lotto():
    url = "https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip"
    tmp_dir = "/content/tmp_lotto"
    os.makedirs(tmp_dir, exist_ok=True)

    r = requests.get(url, timeout=60)
    if r.status_code != 200:
        raise RuntimeError(f"Download fehlgeschlagen. Status: {r.status_code}")

    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        z.extractall(tmp_dir)

    txt_path = os.path.join(tmp_dir, "lotto.txt")
    if not os.path.exists(txt_path):
        raise RuntimeError("lotto.txt nicht gefunden.")

    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    rows = []
    for line in lines[1:]:  # Kopfzeile überspringen
        parts = line.split()
        if len(parts) < 10:
            continue

        tag, monat, jahr = parts[0], parts[1], parts[2]
        rest = parts[3:]

        dt = datetime(int(jahr), int(monat), int(tag))
        if dt >= datetime(2013, 5, 4):
            datum = dt.strftime("%d.%m.%Y")
            rest7 = rest[:7]  # 6 Zahlen + Superzahl
            if len(rest7) == 7:
                rows.append([datum] + rest7)

    df = pd.DataFrame(rows, columns=["Datum","Zahl1","Zahl2","Zahl3","Zahl4","Zahl5","Zahl6","Superzahl"])
    for c in ["Zahl1","Zahl2","Zahl3","Zahl4","Zahl5","Zahl6","Superzahl"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df.dropna().reset_index(drop=True)

def preprocess_df(df):
    df_numbers = df.drop(["Datum", "Superzahl"], axis=1).copy()
    df_sorted = df_numbers.apply(lambda row: pd.Series(sorted(row.astype(int).tolist())), axis=1)
    return df_sorted

def create_training_data(df_sorted, window_length):
    scaler = StandardScaler().fit(df_sorted)
    transformed_df = scaler.transform(df_sorted)
    n_rows, n_feat = df_sorted.shape

    X = np.empty([n_rows - window_length, window_length, n_feat], dtype=float)
    y = np.empty([n_rows - window_length, n_feat], dtype=float)

    for i in range(0, n_rows - window_length):
        X[i] = transformed_df[i : i + window_length, 0 : n_feat]
        y[i] = transformed_df[i + window_length : i + window_length + 1, 0 : n_feat]

    return X, y, scaler

def create_model(input_shape, learning_rate=0.001):
    model = Sequential()
    model.add(Bidirectional(LSTM(240, input_shape=input_shape, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Bidirectional(LSTM(240, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(Dense(49))
    model.add(Dense(input_shape[1]))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["accuracy"])
    return model

def predict_last_window(model, df_sorted, scaler, window_length):
    to_predict = df_sorted.tail(window_length)
    if to_predict.shape[0] < window_length:
        padding = np.zeros((window_length - to_predict.shape[0], to_predict.shape[1]))
        to_predict = np.vstack((padding, to_predict))
    else:
        to_predict = np.array(to_predict)

    scaled_to_predict = scaler.transform(to_predict)
    y_pred = model.predict(np.array([scaled_to_predict]), verbose=0)
    predicted = scaler.inverse_transform(y_pred).astype(int)[0]
    return np.clip(predicted, 1, 49)

def get_url_data():
    """Liest userId und origin URL-Parameter via JavaScript aus."""
    data = {"userId": None, "origin": BACKEND_URL_DEFAULT}
    try:
        from google.colab import output
        js = """
        (function() {
          return window.location.search || "";
        })();
        """
        q = output.eval_js(js)
        if q:
            params = up.parse_qs(q.lstrip("?"))
            if "userId" in params:
                data["userId"] = params["userId"][0]
            if "origin" in params:
                data["origin"] = params["origin"][0]
    except Exception:
        pass
    return data

def send_to_backend(backend_url, payload):
    r = requests.post(f"{backend_url}/api/colab/update", json=payload, timeout=30)
    return r


class UIProgressCallback(Callback):
    def __init__(self, total_epochs, log_fn, progress_widget, base_progress=45, span=35):
        super().__init__()
        self.total_epochs = total_epochs
        self.log_fn = log_fn
        self.progress_widget = progress_widget
        self.base = base_progress
        self.span = span

    def on_train_begin(self, logs=None):
        self.log_fn("Training gestartet…")

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        ep = epoch + 1
        frac = ep / float(self.total_epochs)
        self.progress_widget.value = min(100, int(self.base + self.span * frac))


# ----------------------------
# Runner (wird vom Button aufgerufen)
# ----------------------------
def run_pipeline(_=None):
    progress.value = 0
    tabs.selected_index = 0  # "Ergebnis"
    with out_result:
        clear_output()
    log_area.value = ""

    ui_set_status("Starte…", "info")
    log("Starte Pipeline…")

    try:
        # 1. URL Parameter prüfen
        url_data = get_url_data()
        user_id = url_data["userId"]
        backend_url = url_data["origin"]

        if user_id:
            log(f"👤 User erkannt: ID {user_id}")
        else:
            log("ℹ️ Kein User gefunden (Globaler Modus)")

        progress.value = 5
        ui_set_status("1/5 Lade Lotto-Daten (ZIP) und parse lotto.txt…", "info")
        log("1/5 Download + Parse lotto.txt…")
        df_raw = download_and_parse_lotto()

        with out_result:
            print("✅ Daten geladen:", df_raw.shape)
            display(df_raw.head())

        progress.value = 20
        ui_set_status("2/5 Vorverarbeitung und Trainingsdaten erstellen…", "info")
        log("2/5 Vorbereitung…")
        df_sorted = preprocess_df(df_raw)
        X, y, scaler = create_training_data(df_sorted, model_params["window_length"])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=24)

        progress.value = 35
        ui_set_status("3/5 Modell erstellen…", "info")
        input_shape = (model_params["window_length"], X.shape[2])
        model = create_model(input_shape)

        model_params_runtime = dict(model_params)
        model_params_runtime.update({
            "layers": len(model.layers),
            "optimizer": str(model.optimizer),
        })

        progress.value = 45
        ui_set_status(f"4/5 Training läuft… ({model_params_runtime['epochs']} Epochen)", "warn")
        log(f"4/5 Training Deep Learning Models…")

        cb = UIProgressCallback(
            total_epochs=model_params_runtime["epochs"],
            log_fn=log,
            progress_widget=progress,
            base_progress=45,
            span=35
        )

        history = model.fit(
            x=X_train,
            y=y_train,
            batch_size=model_params_runtime["batch_size"],
            epochs=model_params_runtime["epochs"],
            verbose=0,
            callbacks=[cb]
        )

        progress.value = 82
        ui_set_status("Training fertig. Evaluiere…", "info")
        log("Training beendet. Berechne Scores…")
        train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

        predicted_numbers = predict_last_window(model, df_sorted, scaler, model_params_runtime["window_length"])

        with out_result:
            print("\n📌 Metriken")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
            print(f"Test  Loss: {test_loss:.4f},  Test  Acc: {test_accuracy:.4f}")
            print("\n🎯 Vorhergesagte Zahlen:", predicted_numbers)

        progress.value = 90
        ui_set_status("5/5 Sende Ergebnisse ans Backend…", "info")
        log(f"5/5 Sende an App ({backend_url})...")

        payload = {
            "userId": user_id,  # WICHTIG: User ID
            "numbers": predicted_numbers.tolist() if hasattr(predicted_numbers, "tolist") else list(predicted_numbers),
            "train": {"loss": float(train_loss), "accuracy": float(train_accuracy)},
            "test": {"loss": float(test_loss), "accuracy": float(test_accuracy)},
            "model_params": model_params_runtime,
            "generated_at": int(time.time()),
        }

        resp = send_to_backend(backend_url, payload)

        if resp.ok:
            ui_set_status(f"✅ Gesendet! Schaue jetzt in deine App.", "success")
            log(f"✅ Erfolgreich gesendet! Status: {resp.status_code}")
            progress.value = 100
        else:
            ui_set_status(f"❌ Server-Fehler: {resp.status_code}", "error")
            log(f"❌ Fehler beim Senden: {resp.text[:200]}")
            progress.value = 100

    except Exception as e:
        ui_set_status(f"⚠️ Lauf fehlgeschlagen: <code>{str(e)}</code>", "error")
        log(f"CRITICAL ERROR: {e}")
        progress.value = 100

btn_run.on_click(run_pipeline)
ui_set_status("Bereit. Authentifiziert & Warten auf Start.", "info")